In [1]:
import pymongo
from bson.json_util import dumps
client = pymongo.MongoClient()
mflix = client.sample_mflix
movies = mflix.movies

In [2]:
sam_raimi_cursor = movies.find( { "directors": "Sam Raimi" } )

In [3]:
sam_raimi_cursor

In [4]:
list(sam_raimi_cursor)[0]

{'_id': ObjectId('573a1397f29313caabce843c'),
 'plot': 'Five friends travel to a cabin in the woods, where they unknowingly release flesh-possessing demons.',
 'genres': ['Horror'],
 'runtime': 85,
 'metacritic': 70,
 'rated': 'NC-17',
 'cast': ['Bruce Campbell',
  'Ellen Sandweiss',
  'Richard DeManincor',
  'Betsy Baker'],
 'poster': 'https://m.media-amazon.com/images/M/MV5BODc2MmVjZmUtNjAzMS00MDNiLWIyM2YtOGEzMjg0YjRhMzRmXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SY1000_SX677_AL_.jpg',
 'title': 'The Evil Dead',
 'fullplot': 'As a vacation, five friends travel to a small cabin in a deserted jungle. When their first target has been to reach peace, they should fight restlessly as they find themselves surrounded by an evil force that turns its victims into heartless monsters.',
 'languages': ['English'],
 'released': datetime.datetime(1983, 4, 24, 0, 0),
 'directors': ['Sam Raimi'],
 'writers': ['Sam Raimi'],
 'awards': {'wins': 3, 'nominations': 1, 'text': '3 wins & 1 nomination.'},
 'lastupdate

In [5]:
match_stage = {"$match": { "directors": "Sam Raimi" } }
pipeline = [
    match_stage
]
sam_raimi_aggregation = movies.aggregate( pipeline )

In [6]:
sam_raimi_aggregation

In [7]:
list(sam_raimi_aggregation)[0]

{'_id': ObjectId('573a1397f29313caabce843c'),
 'plot': 'Five friends travel to a cabin in the woods, where they unknowingly release flesh-possessing demons.',
 'genres': ['Horror'],
 'runtime': 85,
 'metacritic': 70,
 'rated': 'NC-17',
 'cast': ['Bruce Campbell',
  'Ellen Sandweiss',
  'Richard DeManincor',
  'Betsy Baker'],
 'poster': 'https://m.media-amazon.com/images/M/MV5BODc2MmVjZmUtNjAzMS00MDNiLWIyM2YtOGEzMjg0YjRhMzRmXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SY1000_SX677_AL_.jpg',
 'title': 'The Evil Dead',
 'fullplot': 'As a vacation, five friends travel to a small cabin in a deserted jungle. When their first target has been to reach peace, they should fight restlessly as they find themselves surrounded by an evil force that turns its victims into heartless monsters.',
 'languages': ['English'],
 'released': datetime.datetime(1983, 4, 24, 0, 0),
 'directors': ['Sam Raimi'],
 'writers': ['Sam Raimi'],
 'awards': {'wins': 3, 'nominations': 1, 'text': '3 wins & 1 nomination.'},
 'lastupdate

In [8]:
sam_raimi_cursor = movies.find(
    { "directors": "Sam Raimi" },
    { "_id": 0, "title": 1, "cast": 1 },
).limit(2)

print(dumps(sam_raimi_cursor, indent=2))

[
  {
    "cast": [
      "Bruce Campbell",
      "Ellen Sandweiss",
      "Richard DeManincor",
      "Betsy Baker"
    ],
    "title": "The Evil Dead"
  },
  {
    "title": "Evil Dead II",
    "cast": [
      "Bruce Campbell",
      "Sarah Berry",
      "Dan Hicks",
      "Kassie Wesley DePaiva"
    ]
  }
]


In [9]:
match_stage = { "$match": { "directors": "Sam Raimi" } }
project_stage = { "$project": { "_id": 0, "title": 1, "cast": 1 } }
limit_stage = { "$limit": 2 }

pipeline = [
    match_stage,
    project_stage,
    limit_stage
]

sam_raimi_aggregation = movies.aggregate( pipeline )

print(dumps(sam_raimi_aggregation, indent=2))

[
  {
    "cast": [
      "Bruce Campbell",
      "Ellen Sandweiss",
      "Richard DeManincor",
      "Betsy Baker"
    ],
    "title": "The Evil Dead"
  },
  {
    "title": "Evil Dead II",
    "cast": [
      "Bruce Campbell",
      "Sarah Berry",
      "Dan Hicks",
      "Kassie Wesley DePaiva"
    ]
  }
]


In [10]:
unwind_stage = { "$unwind": "$directors" }

group_stage = {
    "$group": {
        "_id": {
            "director": "$directors"
        },
        "average_rating": { "$avg": "$imdb.rating" }
    }
}

sort_stage = {
    "$sort": { "average_rating": -1 }
}

# create pipeline from four different stages
pipeline = [
    unwind_stage,
    group_stage,
    sort_stage
]

# aggregate using pipeline
director_ratings = movies.aggregate(pipeline)

# iterate through the resulting cursor
list(director_ratings)[:2]

[{'_id': {'director': 'Sara Hirsh Bordo'}, 'average_rating': 9.4},
 {'_id': {'director': 'Kevin Derek'}, 'average_rating': 9.3}]